In [27]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [28]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch

In [35]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [29]:
PREFIX = 'jun16'
sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
sats_games

['jun16/jun16_4_medium_speed',
 'jun16/jun16_4_no_activity',
 'jun16/jun16_1_undersell_allowed',
 'jun16/jun16_3_no_activity',
 'jun16/jun16_2_undersell_allowed',
 'jun16/jun16_3_medium_speed',
 'jun16/jun16_2_medium_speed',
 'jun16/jun16_1_no_activity',
 'jun16/jun16_0_base',
 'jun16/jun16_4_high_speed',
 'jun16/jun16_2_high_speed',
 'jun16/jun16_1_base',
 'jun16/jun16_0_no_activity',
 'jun16/jun16_4_undersell_allowed',
 'jun16/jun16_0_medium_speed',
 'jun16/jun16_2_no_activity',
 'jun16/jun16_0_undersell_allowed',
 'jun16/jun16_4_base',
 'jun16/jun16_3_high_speed',
 'jun16/jun16_0_high_speed',
 'jun16/jun16_3_undersell_allowed',
 'jun16/jun16_3_base',
 'jun16/jun16_2_base',
 'jun16/jun16_1_medium_speed',
 'jun16/jun16_1_high_speed']

# For running the real dumb test case

In [30]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games if 'hide' not in g and 'imperfect' not in g]
    },
])
print(len(pgrid))

25


In [6]:
# num_iters = 10_000_000
# time_limit_seconds = 3600 * 12
# for config in pgrid:
#     CONFIG = 'cfr_external' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun15external', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [31]:
num_iters = 30_000_000
time_limit_seconds = 3600 * 24
for config in pgrid:
    CONFIG = 'cfr_outcome' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 3_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun16outcome', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 86450
Submitted batch job 86451
Submitted batch job 86452
Submitted batch job 86453
Submitted batch job 86454
Submitted batch job 86455
Submitted batch job 86456
Submitted batch job 86457
Submitted batch job 86458
Submitted batch job 86459
Submitted batch job 86460
Submitted batch job 86461
Submitted batch job 86462
Submitted batch job 86463
Submitted batch job 86464
Submitted batch job 86465
Submitted batch job 86466
Submitted batch job 86467
Submitted batch job 86468
Submitted batch job 86469
Submitted batch job 86470
Submitted batch job 86471
Submitted batch job 86472
Submitted batch job 86473
Submitted batch job 86474
Submitted batch job 86475
Submitted batch job 86476
Submitted batch job 86477
Submitted batch job 86478
Submitted batch job 86479
Submitted batch job 86480
Submitted batch job 86481
Submitted batch job 86482
Submitted batch job 86483
Submitted batch job 86484
Submitted batch job 86485
Submitted batch job 86486
Submitted batch job 86487
Submitted ba

In [80]:
for config in pgrid:
    CONFIG = 'ppo_no_ent_jun14'
    # CONFIG = 'ppo_may31_23' # The YML network config folder for the main jobs
    # BR_CONFIG = 'ppo_may31_23' 
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999_000_000 --eval_zero false --total_timesteps 5_000_000 --dispatch_br false --br_overrides "--total_timesteps 1_000_000"'        
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun14ent_3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)

Submitted batch job 84615
Submitted batch job 84616
Submitted batch job 84617
Submitted batch job 84618
Submitted batch job 84619
Submitted batch job 84620
Submitted batch job 84621
Submitted batch job 84622
Submitted batch job 84623
Submitted batch job 84624
Submitted batch job 84625
Submitted batch job 84626
Submitted batch job 84627
Submitted batch job 84628
Submitted batch job 84629
Submitted batch job 84630
Submitted batch job 84631
Submitted batch job 84632
Submitted batch job 84633
Submitted batch job 84634
Submitted batch job 84635
Submitted batch job 84636
Submitted batch job 84637
Submitted batch job 84638
Submitted batch job 84639
Submitted batch job 84640
Submitted batch job 84641
Submitted batch job 84642
Submitted batch job 84643
Submitted batch job 84644
Submitted batch job 84645
Submitted batch job 84646
Submitted batch job 84647
Submitted batch job 84648
Submitted batch job 84649
Submitted batch job 84650
Submitted batch job 84651
Submitted batch job 84652
Submitted ba

In [37]:
!squeue --format="%.18i %.90j %.2t .%.10M %.6D %R"

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)
             86450                       jun16_jun16_4_medium_speed-cfr_outcomemccfr_outcome-100_jun16outcome  R .   3:59:08      1 ip-10-255-1-169
             86452                       jun16_jun16_4_medium_speed-cfr_outcomemccfr_outcome-102_jun16outcome  R .   5:17:13      1 ip-10-255-1-242
             86457                  jun16_jun16_1_undersell_allowed-cfr_outcomemccfr_outcome-101_jun16outcome  R .   6:21:46      1 ip-10-255-1-134
             86465                       jun16_jun16_3_medium_speed-cfr_outcomemccfr_outcome-100_jun16outcome  R .   4:23:40      1 ip-10-255-1-179
             86467                       jun16_jun16_3_medium_speed-cfr_outcomemccfr_outcome-102_jun16outcome  R .   2:50:06      1 ip-10-255-1-153
             86469                       jun16_jun16_2_medium_speed-cfr_outcomemccfr_outcome-101_jun16outcome  

In [36]:
!squeue | wc -l

17


In [ ]:
!squeue | grep "may30" | wc -l

In [3]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()